In [2]:
user = "Alex"
pathtorepo = "C:\\Users\\" * user * "\\Desktop\\"

using Pkg
Pkg.activate(pathtorepo * "dynamical-systems\\env\\integrate\\")
include("C:\\Users\\" * user * "\\Desktop\\dynamical-systems\\system.jl")

  Activating project at `C:\Users\Alex\Desktop\dynamical-systems\env\integrate`

HR_mem (generic function with 1 method)

In [3]:
using StaticArrays, DifferentialEquations, DynamicalSystems, Statistics
using CairoMakie

In [4]:
function HR_mem(u, p, t)
    function sigma(x)
        return 1.0 / ( 1.0 + exp( -10.0 * ( x  - ( - 0.25 ) ) ) )
    end
    memristor(z, k1_me, k2_me) = k1_me + k2_me * z^2

    a, b, c, d, s, xr, r,  I, vs, k1, k2, k1_me, k2_me  = p
    x1, y1, z1, x2, y2, z2, z = u
    
    du1 = y1 + b * x1 ^ 2 - a * x1 ^3 - z1 + I - k1 * ( x1 - vs ) * sigma(x2) + memristor(z, k1_me, k2_me)*(x2 - x1)
    du2 = c - d * x1 ^2 - y1
    du3 = r * ( s * ( x1 - xr ) - z1 )
    
    du4 = y2 + b * x2 ^ 2 - a * x2 ^3 - z2 + I - k2 * ( x2 - vs ) * sigma(x1) + memristor(z, k1_me, k2_me)*(x1 - x2)
    du5 = c - d * x2 ^2 - y2
    du6 = r * ( s * ( x2 - xr ) - z2 )

    du7 = x1 - x2
    
    return SVector(du1, du2, du3, du4, du5, du6, du7)
end

function save_tofile(name, Λs, u0s)
    nameLS = "LSs"*  name * ".jld"
    nameu0s = "u0s"*  name * ".jld"
    save(nameLS, "data", Λs)
    save(nameu0s, "data", u0s)
end

function output(idx, parameter, u0)
    println("index: $idx; k2_me: $parameter");flush(stdout)
    println("u0: $u0");flush(stdout)
end
function output(Λ, u0)
    println("Λ: $Λ");flush(stdout)
    println("last point: $u0");flush(stdout)
end
function separate()
    println("----------------");flush(stdout)
    println("  ");flush(stdout)
end

separate (generic function with 1 method)

In [5]:
a = 1.0; b = 3.0; c = 1.0; d = 5.0;
xr = -1.6; r = 0.01; s = 5.0; I = 4.0; xv = 2.0;
k1= -0.17; k2 = -0.17;

k1_me = 0.20
k2_me = 0.0
u0_start = [-1.5, 0.0, 0.0, -2.5, 0.0, 0.0, 0.0]
p = [a, b, c, d, s, xr, r, I, xv, k1, k2, k1_me, k2_me];

In [6]:
tt = 5000
t = 5000
tstep = 0.001
integ_set = (alg = RK4(), adaptive = false, dt = tstep)

(alg = RK4(stage_limiter! = trivial_limiter!, step_limiter! = trivial_limiter!, thread = static(false)), adaptive = false, dt = 0.001)

In [7]:
u0 = [-1.5, 0.0, 0.0, -2.5, 0.0, 0.0, 0.0]
k2me_start = 0.0
k2me_end = 1.0
k2me_length = 300

k2me_range = range(k2me_start, k2me_end, length = k2me_length)

In [ ]:
Λs =  zeros(length(k2me_range), 7)
u0s = zeros(length(k2me_range), 7);

In [ ]:
filename = "100 inhibitory k1_me=0.2 quasi"

In [24]:
k2me_range[128]

0.42474916387959866

In [25]:
u0s[128, :]

7-element Vector{Float64}:
 -0.6314695406268608
 -1.7760250192175937
  3.5208160577791396
 -0.6314695406268608
 -1.7760250192175937
  3.5208160577791396
 57.58493480354987

In [1]:
Λs[127, :]

UndefVarError: UndefVarError: `Λs` not defined

In [10]:
for (idx, k2me_) in enumerate(k2me_range)

    if idx == 1
        global u0 = u0_start
    end

    #output(idx, k2me_, u0)

    p = [a, b, c, d, s, xr, r, I, xv, k1, k2, k1_me, k2me_];
    ds = CoupledODEs(HR_mem, u0, p, diffeq = integ_set)

    tr, _ = trajectory(ds, t; Δt = 0.01, Ttr = tt)
    u0 = tr[end]; ds = CoupledODEs(HR_mem,  u0, p, diffeq = integ_set)
    Λ = lyapunovspectrum(ds, t)
    Λ = sort(Λ)
    Λs[idx, :] = Λ
    u0s[idx, :] = u0
    
    #output(Λs[idx, :], u0s[idx, :])
    #separate()
    
    save_tofile(filename, Λs, u0s)
    
end

┌ Warning: Instability detected. Aborting
└ @ SciMLBase C:\Users\Alex\.julia\packages\SciMLBase\KcGs1\src\integrator_interface.jl:606


┌ Warning: Instability detected. Aborting
└ @ SciMLBase C:\Users\Alex\.julia\packages\SciMLBase\KcGs1\src\integrator_interface.jl:606


┌ Warning: Instability detected. Aborting
└ @ SciMLBase C:\Users\Alex\.julia\packages\SciMLBase\KcGs1\src\integrator_interface.jl:606


┌ Warning: Instability detected. Aborting
└ @ SciMLBase C:\Users\Alex\.julia\packages\SciMLBase\KcGs1\src\integrator_interface.jl:606
┌ Warning: Instability detected. Aborting
└ @ SciMLBase C:\Users\Alex\.julia\packages\SciMLBase\KcGs1\src\integrator_interface.jl:606


┌ Warning: Instability detected. Aborting
└ @ SciMLBase C:\Users\Alex\.julia\packages\SciMLBase\KcGs1\src\integrator_interface.jl:606


...

┌ Warning: Instability detected. Aborting
└ @ SciMLBase C:\Users\Alex\.julia\packages\SciMLBase\KcGs1\src\integrator_interface.jl:606


┌ Warning: Instability detected. Aborting
└ @ SciMLBase C:\Users\Alex\.julia\packages\SciMLBase\KcGs1\src\integrator_interface.jl:606


┌ Warning: Instability detected. Aborting
└ @ SciMLBase C:\Users\Alex\.julia\packages\SciMLBase\KcGs1\src\integrator_interface.jl:606


┌ Warning: Instability detected. Aborting
└ @ SciMLBase C:\Users\Alex\.julia\packages\SciMLBase\KcGs1\src\integrator_interface.jl:606


┌ Warning: Instability detected. Aborting
└ @ SciMLBase C:\Users\Alex\.julia\packages\SciMLBase\KcGs1\src\integrator_interface.jl:606
